In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn import GraphConv


/ranjan/SubgraphTransformer/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dgl.data import CoraGraphDataset

dataset = CoraGraphDataset()
g = dataset[0]


  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [3]:
class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes, dropout=0.5):
        super().__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)
        self.dropout = dropout

    def forward(self, g, x):
        # First GCN layer
        x = self.conv1(g, x)
        x = F.relu(x)
        # Dropout helps regularize
        x = F.dropout(x, p=self.dropout, training=self.training)
        
        # Second GCN layer
        x = self.conv2(g, x)
        return x


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
g = g.to(device)

features = g.ndata['feat'].to(device)
labels = g.ndata['label'].to(device)
train_mask = g.ndata['train_mask'].to(device)
val_mask = g.ndata['val_mask'].to(device)
test_mask = g.ndata['test_mask'].to(device)

in_feats = features.shape[1]
num_classes = dataset.num_classes
hidden_feats = 64

model = GCN(in_feats, hidden_feats, num_classes, dropout=0.5).to(device)

# Typical Adam optimizer with weight decay
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=5e-4)

num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    logits = model(g, features)  # [N, num_classes]
    loss = F.cross_entropy(logits[train_mask], labels[train_mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Evaluate
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
        pred = logits.argmax(dim=1)
        
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

    print(
        f"Epoch {epoch:03d} | "
        f"Loss {loss.item():.4f} | "
        f"Train Acc {train_acc.item():.4f} | "
        f"Val Acc {val_acc.item():.4f} | "
        f"Test Acc {test_acc.item():.4f}"
    )


Epoch 000 | Loss 1.9462 | Train Acc 0.3500 | Val Acc 0.2220 | Test Acc 0.2180
Epoch 001 | Loss 1.9335 | Train Acc 0.7143 | Val Acc 0.3600 | Test Acc 0.3730
Epoch 002 | Loss 1.9193 | Train Acc 0.7357 | Val Acc 0.4180 | Test Acc 0.4370
Epoch 003 | Loss 1.9027 | Train Acc 0.6643 | Val Acc 0.4020 | Test Acc 0.4020
Epoch 004 | Loss 1.8803 | Train Acc 0.6500 | Val Acc 0.4040 | Test Acc 0.4260
Epoch 005 | Loss 1.8587 | Train Acc 0.7429 | Val Acc 0.4580 | Test Acc 0.4840
Epoch 006 | Loss 1.8399 | Train Acc 0.8214 | Val Acc 0.5160 | Test Acc 0.5530
Epoch 007 | Loss 1.8111 | Train Acc 0.8714 | Val Acc 0.5860 | Test Acc 0.5870
Epoch 008 | Loss 1.7872 | Train Acc 0.8929 | Val Acc 0.6220 | Test Acc 0.6240
Epoch 009 | Loss 1.7691 | Train Acc 0.9071 | Val Acc 0.6500 | Test Acc 0.6480
Epoch 010 | Loss 1.7306 | Train Acc 0.9357 | Val Acc 0.6720 | Test Acc 0.6810
Epoch 011 | Loss 1.6984 | Train Acc 0.9357 | Val Acc 0.6780 | Test Acc 0.6980
Epoch 012 | Loss 1.6583 | Train Acc 0.9357 | Val Acc 0.6880 | Te